In [1]:
import sys
import os
import json

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
# Liquidez simple USDC/WHBAR: ±5% ticks, 50/50 por valor; mint con msg.value (patrón UI)
import json, requests, math
from adapters.SaucerSwap.adapter.SaucerSwapAdapter import SaucerSwapAdapter

CONFIG_PATH = "/root/Repositorios/idl/adapters/SaucerSwap/config/hedera.saucerswap.yaml"
USDC = "0.0.456858"
FEE_BPS = 1500
RANGE_PCT = 0.05
FLEX_BPS = 0
SEND = True

adapter = SaucerSwapAdapter(CONFIG_PATH)

# Pool USDC/WHBAR
base = adapter.config.api_base.rstrip("/")
pools = requests.get(base + "/v2/pools", headers={"x-api-key": adapter.api_key}, timeout=30).json()
whbar_id = adapter._whbar_token_id()
pool = next(p for p in pools if int(p.get("fee", 0)) == FEE_BPS and { (p.get("tokenA") or {}).get("id"), (p.get("tokenB") or {}).get("id") } == {USDC, (whbar_id or "") })
POOL_ID = pool["id"]
tA, tB = pool["tokenA"], pool["tokenB"]

dec_usdc = int((tA if tA["id"] == USDC else tB)["decimals"])
dec_wh   = int((tA if tA["id"] == (whbar_id or "") else tB)["decimals"])
px_wh    = float((tA if tA["id"] == (whbar_id or "") else tB)["priceUsd"])  # 1 WHBAR ~ 1 HBAR

# Ticks ±5% y rounding a spacing
ratio = adapter.get_pool_ratio(POOL_ID)
tick_current = int(ratio["tickCurrent"])
tick_lower = tick_current - int(abs(tick_current) * RANGE_PCT)
tick_upper = tick_current + int(abs(tick_current) * RANGE_PCT)
pool_evm = adapter._hts_to_evm(pool.get("contractId", ""))
ts = adapter._pool_tick_spacing(pool_evm)
tick_lower = adapter._round_tick_to_spacing(tick_lower, ts, "floor")
tick_upper = adapter._round_tick_to_spacing(tick_upper, ts, "ceil")
if tick_lower >= tick_upper:
    tick_upper = tick_lower + ts

# Presupuesto en USD definido por el usuario y reparto 50/50 por valor (máxima flexibilidad: mins=0)
TOTAL_USD = 10.0  # <-- ajusta tu presupuesto en USD
wh_id = whbar_id or ""
# Convertir presupuesto a cantidades deseadas (no restringimos mínimos; la pool decide el consumo)
trg0_usd = 0.5 * TOTAL_USD
trg1_usd = 0.5 * TOTAL_USD
amount0_des = int(math.floor(trg0_usd * (10**dec_usdc)))
amount1_des = int(math.floor((trg1_usd / max(1e-9, px_wh)) * (10**dec_wh)))

# Balances para clamping
bals_now = adapter.get_balances([USDC])
bal_usdc_raw = int(bals_now.get(USDC, {}).get("raw", 0))
# clamp USDC; WHBAR lo aporta msg.value en el mint
amount0_des = min(amount0_des, bal_usdc_raw)

# limitar por maxLiquidityPerTick
from eth_utils import keccak as _keccak
sel_maxL = _keccak(text="maxLiquidityPerTick()")[:4]
try:
    maxL_hex = adapter._call_rpc("eth_call", [{"to": pool_evm, "data": "0x" + sel_maxL.hex()}, "latest"]) or "0x0"
    L_max = int(maxL_hex, 16)
except Exception:
    L_max = None
# calcular L y escalar si excede
sqrtP_x96 = adapter._sqrt_ratio_x96_from_tick(tick_current)
sqrtA_x96 = adapter._sqrt_ratio_x96_from_tick(tick_lower)
sqrtB_x96 = adapter._sqrt_ratio_x96_from_tick(tick_upper)
if L_max:
    L = adapter._liquidity_for_amounts(sqrtP_x96, sqrtA_x96, sqrtB_x96, amount0_des, amount1_des)
    if L > L_max and L > 0:
        scale = max(1e-6, L_max / L)
        amount0_des = max(0, int(amount0_des * scale))
        amount1_des = max(0, int(amount1_des * scale))

# Mint
quote = adapter.liquidity_quote_by_ticks(
    tokenA=(USDC if tA["id"] == USDC else wh_id), tokenB=(wh_id if tB["id"] == wh_id else USDC), fee_bps=FEE_BPS,
    tick_lower=tick_lower, tick_upper=tick_upper,
    amount0_desired=amount0_des if tA["id"] == USDC else amount1_des,
    amount1_desired=amount1_des if tB["id"] == wh_id else amount0_des,
    slippage_bps=0,
)
# Asociar LP NFT antes del mint
try:
    if getattr(adapter, "_lp_nft_id", None):
        adapter.associate_execute(adapter._lp_nft_id)
except Exception as _aexc:
    print("associate LP NFT error:", _aexc)

prep = adapter.liquidity_prepare(quote, deadline_s=300)
# Simplificación: no gestionamos HBAR para wrap; el adapter fija msg.value (UI)

try:
    if SEND:
        res = adapter.liquidity_send(prep, wait=True)
        print(json.dumps(res, indent=2, default=str))
    else:
        print(json.dumps({"prep": prep}, indent=2, default=str))
except Exception:
    # si falla, unwrap WHBAR -> HBAR como salvaguarda
    try:
        sweep = adapter._whbar_sweep_unwrap(adapter.evm_address)
        print(json.dumps({"unwrap": sweep}, indent=2, default=str))
    except Exception as exc2:
        print(json.dumps({"unwrap_error": str(exc2)}, indent=2))
    raise


2025-09-18 17:52:51,764 INFO [SaucerSwapAdapter] SaucerSwapAdapter inicializado (network=mainnet, account_id=0.0.9637418, evm=0x3845BbD7047E65A1336a42B647487f5dc4baD93A)
2025-09-18 17:52:54,551 INFO [SaucerSwapAdapter] liq_quote: tokens HTS tA=0.0.456858 tB=0.0.1456986 fee_bps=1500
2025-09-18 17:52:54,923 INFO [SaucerSwapAdapter] liq_quote: ticks [57000,63030] amounts(desired)=(5000000,2020318668) mins=(0,0) poolId=9
2025-09-18 17:52:57,700 INFO [SaucerSwapAdapter] liq_prep: npm=0x00000000000000000000000000000000003ddbb9 fee_bps=1500 ticks=[57000,63030] amounts(desired)=(5000000,2020318668) mins=(0,0) recipient=0x3845BbD7047E65A1336a42B647487f5dc4baD93A
2025-09-18 17:53:05,768 INFO [SaucerSwapAdapter] get_mint_fee: wei=203120190000000000 source=master_chef
2025-09-18 17:53:06,523 INFO [SaucerSwapAdapter] liq_prep: gas=1375000 value=0x11b31c3fb23f69c00 canSend=False notes=NPM posiblemente no asociado a 0.0.456858 (no bloqueante); mint via multicall([mint, refundETH]); saldo insuficiente

{
  "steps": [
    {
      "mint": {
        "txHash": "0x7982d1d083316d0332a9aa8ef1a2bc676cb350cd783dde9d46cc5111be0e9f24",
        "receipt": {
          "blockHash": "0x33940aaec0c0880440137127dd85b4a204ec076d98cba103547e3d39ac1d17a6",
          "blockNumber": "0x50d37cd",
          "from": "0x3845bbd7047e65a1336a42b647487f5dc4bad93a",
          "to": "0x00000000000000000000000000000000003ddbb9",
          "cumulativeGasUsed": "0x195460",
          "gasUsed": "0x10c8e0",
          "contractAddress": "0x00000000000000000000000000000000003ddbb9",
          "logs": [
            {
              "address": "0x00000000000000000000000000000000003ddbb9",
              "blockHash": "0x33940aaec0c0880440137127dd85b4a204ec076d98cba103547e3d39ac1d17a6",
              "blockNumber": "0x50d37cd",
              "data": "0x00000000000000000000000000000000000000000000000000000000004c4b40",
              "logIndex": "0x0",
              "removed": false,
              "topics": [
                "0x